# Import libraries

In [1]:
import jupyros as jr
import rospy
import ipywidgets as widgets

from nav_msgs.msg import Odometry
from assignment_2_2022.msg import Pos_and_Vel
from assignment_2_2022.srv import Num_Goal_rc
from sensor_msgs.msg import LaserScan
import actionlib
import actionlib.msg
import assignment_2_2022.msg

import matplotlib.pyplot as plt
import tf
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation


ModuleNotFoundError: No module named 'assignment_2_2022'

# Variables and initialization Node A

In [ ]:
global goal
global client
global publisher
global msg
global num_c
global num_r

num_c = 0
num_r = 0

#initialize the node
rospy.init_node('RT2_nodeA')

# Set Publisher  on the topic /pos_and_vel

In [ ]:
publisher = rospy.Publisher("/pos_and_vel", Pos_and_Vel, queue_size = 10)

# Callback function

In [ ]:
def callback(msg):
    
    pos_vel = Pos_and_Vel()
    
    pos_vel.x = msg.pose.pose.position.x
    pos_vel.y = msg.pose.pose.position.y
    pos_vel.vel_x = msg.twist.twist.linear.x
    pos_vel.vel_y = msg.twist.twist.linear.y
    
    publisher.publish(pos_vel)

# Set Subscriber to the topic /odom

In [ ]:
jr.subscribe("/odom", Odometry, callback)

# Set Action Client 

In [ ]:
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)

#wait for server
client.wait_for_server()

# Widgets interface for setting/canceling goal

In [ ]:
goal = assignment_2_2022.msg.PlanningGoal()

# Goal coordinates
x = widgets.FloatText(value=0 ,step=0.5, description = "x:", orientation='horizontal')
y = widgets.FloatText(value=0 ,step=0.5, description = "y:", orientation='horizontal')

# Widgets Buttons
send_button = Button(description='Send goal',tooltip='Send goal',layout=Layout(width='50%', height='100px', grid_area='b1'),button_style='info')
cancel_button = Button(description='Cancel goal',tooltip='Cancel goal',layout=Layout(width='50%',height='100px', grid_area='b2'),button_style='danger')


# if send button is clicked 
def on_button_send_clicked(b):
    
    goal.target_pose.pose.position.x = x.value
    goal.target_pose.pose.position.y = y.value
    
    #update goal list
    goal_list.value = goal_list.value + "Goal x: " + str(x.value) + "  Goal y: " + str(y.value)
    
    #send the goal returned by set_goal()
    client.send_goal(goal)
    
    x.disabled = True
    y.disabled = True
    send_button.disabled = True
    cancel_button.disabled = False
    

send_button.on_click(on_button_send_clicked)


def on_cancel_button_clicked(b):
    client.cancel_goal()
    
cancel_button.on_click(on_cancel_button_clicked)
        

widgets.HBox([widgets.VBox([x, y]), widgets.VBox([send_button, cancel_button])])



# Feedback

In [ ]:
global reached_list
global not_reached_list

reached_list = [0]
not_reached_list = [0]

send_button, cancel_button

def update_goals_lists():
    #global n_reached, n_not_reached
    reached_list.append(num_r)
    not_reached_list.append(num_c)

def checkGoalFeedback(data):
    
    #global n_reached, n_not_reached
    
    if data.feedback.stat == "Target reached!":
        
        x.disabled = False
        y.disabled = False
        
        send_button.disabled = False
        cancel_button.disabled = True
        
        #status.value = "Goal reached!"
        
        goal_list.value = goal_list.value + "    Goal reached!\n"
        
        num_r += 1
        
        update_goals_lists()
        
    elif data.feedback.stat == "Target cancelled!":
        
        x.disabled = False
        y.disabled = False
        
        send_button.disabled = False
        cancel_button.disabled = True
        
        #status.value = "Goal cancelled!"
        
        goal_list.value = goal_list.value + "    Goal cancelled!\n"
        
        num_c += 1
        
        update_goals_lists()

# Set Subscriber for feedback

In [ ]:
jr.subscribe("/reaching_goal/feedback", assignment_2_2022.msg.PlanningActionFeedback, checkGoalFeedback)

# Live pos and vel

In [ ]:
posx = widgets.FloatText(description = "Pos x:", disabled = True)
posy = widgets.FloatText(description = "Pos y:", disabled = True)
velx = widgets.FloatText(description = "Vel x:", disabled = True)
vely = widgets.FloatText(description = "Vel y:", disabled = True)
    
widgets.HBox([widgets.VBox([posx, posy]), widgets.VBox([velx, vely])])

# Data Visualization functions

In [ ]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [] , []
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln
    def vis_callback(self, data):
        self.y_data.append(data.y)
        self.x_data.append(data.x)
        posx.value = data.x
        posy.value = data.y
        velx.value = data.vel_x
        vely.value = data.vel_y
        
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

# Visualize Data

In [ ]:
get_ipython().run_line_magic('matplotlib', 'widget')
vis = Visualiser()
sub = jr.subscribe('/pos_and_vel', Pos_and_Vel, vis.vis_callback)

ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

# Display goal list

In [ ]:
goal_list = widgets.Textarea(value = "Goal list:\n", disabled = True, style = dict(text_color = 'red') )

display(goal_list)

# Laser function

In [ ]:
rmin = widgets.FloatText(description = "Distance:", disabled = True)
ang = widgets.FloatText(description = "Angle:", disabled = True)

def laserCallback(scan):
    min_range = 100
    angle = 100
    for at, x in enumerate(scan.ranges):
        if x < min_range and x > scan.range_min:
            min_range = x
            angle = scan.angle_min + scan.angle_increment * at
    
    rmin.value = min_range
    ang.value = angle
    
widgets.HBox([rmin, ang], description = "Distance from obstacle")

# Set Subscriber to topic /scan

In [ ]:
jr.subscribe('/scan', LaserScan, laserCallback)

# Display obstacle distance with laser

In [ ]:
get_ipython().run_line_magic('matplotlib', 'widget')

button_update = widgets.Button(description = "Update graph!", button_style = "success")

def on_button_update_clicked(b):
    plt.figure(2)
    line1, = plt.plot(reached_list, label="Reached", color = 'green')
    line2, = plt.plot(not_reached_list, label="Not-Reached", color = 'red')
    # Create a legend for the first line.
    first_legend = plt.legend(handles=[line1], loc='lower left')
    # Add the legend manually to the current Axes.
    plt.gca().add_artist(first_legend)
    # Create another legend for the second line.
    plt.legend(handles=[line2], loc='lower right')
    plt.show()
    
button_update.on_click(on_button_update_clicked)
display(button_update)